In [3]:
import sqlite3
import pandas as pd

# Helper function

In [37]:
def formed_col_sql(columns):
    string_output = ""
    for column in columns[:-1]:
        string_output += f"{column} TEXT, "
    string_output += f"{columns[-1]} TEXT"
    return string_output

In [71]:
def insert_data_into_table(table_name, data_pandas, cursor):
    string_columns = ''
    string_values = ''
    for col in data_pandas.columns[:-1]:
        string_columns += f'{col}, '
    string_columns += f'{data_pandas.columns[-1]}'
    string_columns = f'{table_name}_key, '+ string_columns
    string_request = f"""INSERT INTO {table_name} ({string_columns}) VALUES ({len(data_pandas.columns)*'?, ' + '?'})"""
    for idx in range(len(data_pandas)):
        insert_data = (idx, *data_pandas.iloc[idx].values)
        cursor.execute(string_request, insert_data)

# Constant

In [6]:
dataset_sql_name = 'dataset_sql'
symptom_data_sql_name = 'symptom_data_sql'
symptom_precaution_sql_name = 'symptom_precaution_sql'
symptom_severity_sql_name = 'symptom_severity_sql'

# Load pandas data

In [7]:
# загрузка датасетов
dataset = pd.read_csv('data/dataset.csv')
symptom_data = pd.read_csv('data/symptom_Description.csv')
symptom_precaution = pd.read_csv('data/symptom_precaution.csv')
symptom_severity = pd.read_csv('data/Symptom-severity.csv')

# Make con and execute

In [4]:
con = sqlite3.connect('dataset.db')

In [5]:
cur = con.cursor()

In [44]:
cur.execute(f"""
    CREATE TABLE IF NOT EXISTS {symptom_data_sql_name} 
        (
            {symptom_data_sql_name}_key INTEGER PRIMARY KEY AUTOINCREMENT,
            Disease	TEXT,
            Description TEXT
        )""")

cur.execute(f"""CREATE TABLE IF NOT EXISTS {symptom_precaution_sql_name}
        (
            {symptom_precaution_sql_name}_key INTEGER PRIMARY KEY AUTOINCREMENT,
            {formed_col_sql(symptom_precaution.columns)}
        )""")

cur.execute(f"""CREATE TABLE IF NOT EXISTS {symptom_severity_sql_name}
        (
            {symptom_severity_sql_name}_key INTEGER PRIMARY KEY AUTOINCREMENT,
            {formed_col_sql(symptom_severity.columns)}
        )""")
    
cur.execute(f"""CREATE TABLE IF NOT EXISTS {dataset_sql_name}
        (
            {dataset_sql_name}_key INTEGER PRIMARY KEY AUTOINCREMENT,
            {formed_col_sql(dataset.columns)}
        )""")

In [75]:
# Fill tables
insert_data_into_table(dataset_sql_name, dataset, cur)
insert_data_into_table(symptom_data_sql_name, symptom_data, cur)
insert_data_into_table(symptom_precaution_sql_name, symptom_precaution, cur)
insert_data_into_table(symptom_severity_sql_name, symptom_severity, cur)

con.commit()